## Manual publication DB insertion from raw text using syntax features
### Publications and conferences of Dr. CHIRA Camelia, Profesor Universitar
#### http://www.cs.ubbcluj.ro/~cchira

In [1]:
class HelperMethods:
    @staticmethod
    def IsDate(text):
#         print("text")
#         print(text)
        for c in text.lstrip():
            if c not in "1234567890 ":
                return False
        return True

In [2]:
import pandas
import requests

In [3]:
page = requests.get('http://www.cs.ubbcluj.ro/~cchira/publications.html')

In [4]:
data = page.text


In [5]:
from bs4 import BeautifulSoup

In [6]:
soup = BeautifulSoup(data)

In [134]:
def SearchDate(text):
    date_val = re.search('\(?(1|2)(9|0)[0-9]{2}( ?)(\.|,|\))', text)
    try:
        date = date_val.group(0)
    except AttributeError:
        print(text)
    return date.lstrip('\(').rstrip(",. \)")

In [104]:
import re
def GetPublicationTitle(text):
    match = re.search(', ([a-zA-Z ]+-?:?){3,}(,|.)', text)
    try:
        title = match.group(0).lstrip(', ')
    except AttributeError:
        return ""
    date = SearchDate(text)
    return title, date

In [105]:
def GetPublicationTitle_B(text):
#     print("B: ")
#     print(text)
    authors = "".join(text.split('.,'))
    title_val = re.search('([a-zA-Z]+ ?-?)+(,)', text)
    title = title_val.group(0)
    date = SearchDate(text)
    return title, date

In [135]:
pubs = []
# print(soup.find_all('div'))
for e in soup.find_all('div', attrs={"class": "section_mine"}):

    lines = e.find_all("p")
    for line in lines:
#         print(line)
        try:
            title = line.find_all('a')
            title = title[0].contents[0]
            date = SearchDate(line.text)
        except:
            title, date = GetPublicationTitle(line.text)
        if len(title) < 25 and '.' in title:
            title, date = GetPublicationTitle_B(line.text)
        print("title: ", title)
        print("date: ", date)
        pubs.append((title, date))

title:  Improving Fall Detection Using an On-Wrist Wearable Accelerometer
date:  2018
title:  Key features for the characterization of Android malware families
date:  2017
title:  The generalized traveling salesman problem solved with ant algorithms
date:  2017
title:  MobiContext: A Context-Aware Cloud-Based Venue Recommendation Framework
date:  2017
title:  A parallel evolutionary approach to community detection in complex networks
date:  2017
title:  Characterization of Android Malware Families by a Reduced Set of Static Features,
date:  2017
title:  Gene clustering for time-series microarray with production outputs
date:  2016
title:  Evolutionary community detection in complex and dynamic networks 
date:  2016
title:  A hybrid intelligent recognition system for the early detection of strokes
date:  2015
title:  S4MPLEâSampler for Multiple Protein-Ligand Entities: Methodology and Rigid-Site Docking Benchmarking
date:  2015
title:  Improving Human Activity Recognition and its Appl

# DB Storage (TODO)
Time to store the entries in the `papers` DB table.


![Screenshot](Images/PapersTableSpec.PNG)

In [119]:
import mariadb
import json

In [120]:
with open('../credentials.json', 'r') as crd_json_fd:
    json_text = crd_json_fd.read()
    json_obj = json.loads(json_text)

In [121]:
credentials = json_obj["Credentials"]
username = credentials["username"]
password = credentials["password"]

In [122]:
table_name = "publications_cache"
db_name = "ubbcluj"

In [123]:
mariadb_connection = mariadb.connect(user=username, password=password, database=db_name)
mariadb_cursor = mariadb_connection.cursor()

In [137]:
for paper in pubs:
    
    title = ""
    pub_date = ""
    affiliations = ""
    
    try:
        pub_date = paper[1].lstrip()
        pub_date = str(pub_date) + "-01-01"
        if len(pub_date) != 10:
            pub_date = ""
    except:
        pass
    
    try:
        title = paper[0].lstrip()
    except:
        pass
    
    
    insert_string = "INSERT INTO {0} SET ".format(table_name)
    insert_string += "Title=\'{0}\', ".format(title)
    insert_string += "ProfessorId=\'{0}\', ".format(9)
    if pub_date != "":
        insert_string += "PublicationDate=\'{0}\', ".format(str(pub_date))
    insert_string += "Authors=\'{0}\', ".format("")
    insert_string += "Affiliations=\'{0}\' ".format("")
    print(insert_string)

    try:
        mariadb_cursor.execute(insert_string)
    except mariadb.ProgrammingError as pe:
        print("Error")
        raise pe
    except mariadb.IntegrityError:
        continue

    

INSERT INTO publications_cache SET Title='Improving Fall Detection Using an On-Wrist Wearable Accelerometer', ProfessorId='9', PublicationDate='2018-01-01', Authors='', Affiliations='' 
INSERT INTO publications_cache SET Title='Key features for the characterization of Android malware families', ProfessorId='9', PublicationDate='2017-01-01', Authors='', Affiliations='' 
INSERT INTO publications_cache SET Title='The generalized traveling salesman problem solved with ant algorithms', ProfessorId='9', PublicationDate='2017-01-01', Authors='', Affiliations='' 
INSERT INTO publications_cache SET Title='MobiContext: A Context-Aware Cloud-Based Venue Recommendation Framework', ProfessorId='9', PublicationDate='2017-01-01', Authors='', Affiliations='' 
INSERT INTO publications_cache SET Title='A parallel evolutionary approach to community detection in complex networks', ProfessorId='9', PublicationDate='2017-01-01', Authors='', Affiliations='' 
INSERT INTO publications_cache SET Title='Characte

INSERT INTO publications_cache SET Title='M,', ProfessorId='9', PublicationDate='2006-01-01', Authors='', Affiliations='' 
INSERT INTO publications_cache SET Title='Stigmergic Agents for Solving NP-difficult Problems,', ProfessorId='9', PublicationDate='2006-01-01', Authors='', Affiliations='' 
INSERT INTO publications_cache SET Title='Reinforcing Ant Colony System for the Generalized Traveling Salesman Problem,', ProfessorId='9', PublicationDate='2006-01-01', Authors='', Affiliations='' 
INSERT INTO publications_cache SET Title='A Multi-Agent Approach to Distributed Collaboration,', ProfessorId='9', PublicationDate='2006-01-01', Authors='', Affiliations='' 
INSERT INTO publications_cache SET Title='Development of a Multi-Agent Information Management System,', ProfessorId='9', PublicationDate='2006-01-01', Authors='', Affiliations='' 
INSERT INTO publications_cache SET Title='Chira,', ProfessorId='9', PublicationDate='2005-01-01', Authors='', Affiliations='' 
INSERT INTO publications_c

In [68]:
mariadb_connection.close()

# Conclusion
### In the end, the DB only required ~4 manual modifications (deletions) with this code.
126 rows inserted.

![Screenshot](Images/Finished.PNG)